In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def get_params_amount(model):
    amount = 0
    for _, layer in model.named_children():
        if isinstance(layer, ExpandingLinear):
            for linear in layer.embed_linears:
                amount += linear.weight_values.shape[0]
            amount += layer.weight_values.shape[0]
        elif isinstance(layer, nn.Linear):
            amount += linear.in_features * linear.out_features
    return amount

In [5]:
def get_zero_params_amount(model, eps=1e-8):
    amount = 0
    for _, layer in model.named_children():
        if isinstance(layer, ExpandingLinear):
            for linear in layer.embed_linears:
                amount += linear.weight_values[linear.weight_values.abs() < eps].shape[0]
            amount += layer.weight_values[layer.weight_values.abs() < eps].shape[0]
        elif isinstance(layer, nn.Linear):
            amount += linear.weight[linear.weight.abs() < eps].numel()
    return amount

In [6]:
def get_to_replace_params_amount(ef, model, layer_names, mask, choose_threshold):
    chosen_edges = 0
    for layer_name in layer_names:
        chosen_edges += len(ef.choose_edges_threshold(model, layer_name, choose_threshold, mask)[0])
    return chosen_edges

In [7]:
def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, window_size=5, threshold=0.2, choose_threshold=0.4):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    ef = EdgeFinder(metric, val_loader, device, aggregation_mode='mean')

    replace_epoch = [0]
    val_losses = []
    len_choose = get_model_last_layer(model).count_replaces[0]
    for epoch in range(num_epochs):
        t0 = time.time()
        model.train()
        train_loss = 0
        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss += loss.item()
        
        # if len(replace_epoch) > 1:
        #     for g in optimizer.param_groups:  
                # g['lr'] *= 0.9
        
        train_loss /= len(train_loader)
        train_time = time.time() - t0

        model.eval()
        val_loss = 0
        all_targets = []
        all_preds = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_targets.extend(targets.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_targets, all_preds)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        val_losses.append(val_loss)
        if len(val_losses) > window_size and epoch - replace_epoch[-1] > 8:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                # layer = model.fc0
                # mask = torch.ones_like(layer.weight_values, dtype=bool)
                # len_choose = edge_replacement_func_new_layer(layer, mask, optimizer, val_loader, metric, 0.3, 'mean')

                layer = model.fc0
                mask = torch.ones_like(layer.weight_values, dtype=bool)
                len_choose = edge_replacement_func_new_layer(model, 'fc0', mask, optimizer, choose_threshold, ef)

                wandb.log({'len_choose': len_choose})
                replace_epoch += [epoch]

                # if len(replace_epoch) == 2:
                #     for g in optimizer.param_groups:
                #         g['lr'] *= 100


        params_amount = get_params_amount(model)
        # zero_params_amount = get_zero_params_amount(model)
        layer = model.fc0
        mask = torch.ones_like(layer.weight_values, dtype=bool)
        replace_params = get_to_replace_params_amount(ef, model, ['fc0'], mask, choose_threshold)
        wandb.log({'val loss': val_loss, 'val accuracy': val_accuracy,
                   'train loss': train_loss, 'params amount': params_amount,
                   'params to replace amount': replace_params, 'train time': train_time,
                   'params ratio': (params_amount - replace_params) / params_amount,
                   'lr': optimizer.param_groups[0]['lr'], 'acc amount': val_accuracy / params_amount})

def edge_replacement_func_new_layer(model, layer_name, mask, optim, choose_threshold, ef):
    layer = model.__getattr__(layer_name)
    # ef = EdgeFinder(metric, val_loader, device, aggregation_mode)
    chosen_edges = ef.choose_edges_threshold(model, layer_name, choose_threshold, mask)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if len(chosen_edges[0]) > 0:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
        optim.add_param_group({'params': layer.weight_values})
    print(len(chosen_edges[0]))
    return len(chosen_edges[0])

# def edge_replacement_func_new_layer(model, optim, val_loader, metric, choose_threshold, aggregation_mode='mean', len_choose=None):
#     layer = get_model_last_layer(model)
#     ef = EdgeFinder(metric, val_loader, device, aggregation_mode)
#     vals = ef.calculate_edge_metric_for_dataloader(model, len_choose, False)
#     print("Edge metrics:", vals, max(vals, default=0), sum(vals))
#     chosen_edges = ef.choose_edges_threshold(model, choose_threshold, len_choose)
#     print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
#     layer.replace_many(*chosen_edges)

#     if len(chosen_edges[0]) > 0:
#         optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
#         # optim.add_param_group({'params': layer.weight_values})
#     else:
#         print("Empty metric")

#     return {'max': max(vals, default=0), 'sum': sum(vals), 'len': len(vals), 'len_choose': layer.count_replaces[-1]}

In [8]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [9]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [10]:
criterion = nn.CrossEntropyLoss()
metrics = [
    MagnitudeL2Metric(criterion),
    # SNIPMetric(criterion),
    # GradientMeanEdgeMetric(criterion),
    # PerturbationSensitivityEdgeMetric(criterion),
]
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
)

In [ ]:
train_sparse_recursive(sparse_model, train_loader, val_loader, 64, metrics[0])

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [00:02<00:00, 331.33it/s]


Epoch 1/64, Train Loss: 1.4805, Val Loss: 0.9977, Val Accuracy: 0.8192
589 0.07512755102040816


100%|██████████| 750/750 [00:02<00:00, 332.58it/s]


Epoch 2/64, Train Loss: 0.8088, Val Loss: 0.6843, Val Accuracy: 0.8528
511 0.06517857142857143


...

100%|██████████| 750/750 [00:09<00:00, 78.56it/s]


Epoch 46/64, Train Loss: 0.0850, Val Loss: 0.1370, Val Accuracy: 0.9581
Chosen edges: tensor([[   3,    3,    3,    6,    6,    6,    6,    6,    7,    7,    7,    9,
            0,    3,    6,    7,    7,    7,    7,    7,    9,    0,    0,    1,
            3,    7,    2,    2,    2,    2,    7,    7,    7,    8,    7,    7,
            6,    7,    8,    8,    8,    9,    9,    9,    1,    1,    1,    1,
            2,    2,    2,    2,    2,    3,    3,    3,    3,    3,    4,    4,
            4,    4,    4,    4,    4,    4,    4,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    6,    6,    6,    6,    6,    6,
            6,    8,    8,    8,    9,    9,    9,    9,    9,    9,    9,    9,
            9,    9,    9],
        [ 136,  332,  360,  621,  650,  681,  710,  719,   97,  100,  121,  104,
          791, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159,
         1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1

100%|██████████| 750/750 [00:10<00:00, 69.62it/s]


Epoch 47/64, Train Loss: 0.0833, Val Loss: 0.1354, Val Accuracy: 0.9581
103 0.0014006554523573168


100%|██████████| 750/750 [00:10<00:00, 68.97it/s]


- прунинг по метрике на следующей эпохе после реплейса
